In [ ]:
## mercadolibre brand mapping
## Author : Nitin Bhayana 
## 10 June 2022
## version : 1.0

In [1]:

import numpy as np
from bs4 import BeautifulSoup


from tqdm.notebook import tqdm
import time

import pandas as pd
import psycopg2
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

from tqdm import tqdm
tqdm.pandas()
import time

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
           "Accept-Encoding": "gzip, deflate, br",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "DNT":"1",
           "Connection":"close",
           "Upgrade-Insecure-Requests": "1"}

class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

rawdf=serp_details.drop_duplicates('sku')

rawdf1=rawdf

df_ASIN = rawdf1[["sku"]]
df_ASIN.dropna(inplace=True)
df_ASIN.drop_duplicates(inplace=True)
asinlist = list(df_ASIN['sku'])
len(asinlist)


query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""

pm_data = pd.read_sql_query(query_2, conn[1])



mapped_df=pd.merge(rawdf1,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])
mapped_df

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Enter keystone ids with comma.6406c4d592d04690dfec10c7
 Enter no of days for which data needs brand mapping.
135


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148792\3118012154.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148792\3118012154.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.dropna(inplace=True)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_148792\3118012154.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.drop_dup

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,None,Ldas Disposable Geriatric Qualifral Eg 7 Units,MLB-2168433399,mercadolibre rno 367 serp job,46,MLB-2168433399,BR,Qualifral,None,None
1,None,Qualifral Diaper Package Eg 26 Units,MLB-3031049398,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN
2,None,Qualifral Diaper Package M 30 Units,MLB-3031029093,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN
3,None,Qualifral Diaper Package G 30 Units,MLB-3031124770,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN
4,None,Qualifral Premium MC/8 diaper,MLB-3031304228,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1998,None,100% Acrylic Rain Deflector Several Smoked Car...,MLB-2893416046,mercadolibre rno 367 serp job,46,MLB-2893416046,BR,Ecoflex,None,None
1999,None,Pair of Cleaner Blade Original Model Bosch Aer...,MLB-1510200498,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN
2000,None,100% Acrylic Rain Gutter Several Smoked Cars 4...,MLB-2893386717,mercadolibre rno 367 serp job,46,MLB-2893386717,BR,Ecoflex,None,None
2001,None,Tena Pants Men S/M Underwear EG/EG- 2 Packs,MLB-2655409498,mercadolibre rno 367 serp job,46,NaN,NaN,NaN,NaN,NaN


In [2]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name']]
df2

,channel_id,sku,title,brand_name
0,46,MLB-2168433399,Ldas Disposable Geriatric Qualifral Eg 7 Units,Qualifral
1,46,MLB-3031049398,Qualifral Diaper Package Eg 26 Units,NaN
2,46,MLB-3031029093,Qualifral Diaper Package M 30 Units,NaN
3,46,MLB-3031124770,Qualifral Diaper Package G 30 Units,NaN
4,46,MLB-3031304228,Qualifral Premium MC/8 diaper,NaN
...,...,...,...,...
1998,46,MLB-2893416046,100% Acrylic Rain Deflector Several Smoked Car...,Ecoflex
1999,46,MLB-1510200498,Pair of Cleaner Blade Original Model Bosch Aer...,NaN
2000,46,MLB-2893386717,100% Acrylic Rain Gutter Several Smoked Cars 4...,Ecoflex
2001,46,MLB-2655409498,Tena Pants Men S/M Underwear EG/EG- 2 Packs,NaN


In [3]:
df=pd.DataFrame(columns=['channel_sku_id','brand_name','categories','sold'])
df

,channel_sku_id,brand_name,categories,sold


In [13]:
asinlist = list(df2['sku'])
len(asinlist)

2003

In [12]:
batches=int(len(asinlist)/50 )+1
batches

2

In [14]:
asinlist

['MLB-2168433399',
 'MLB-3031049398',
 'MLB-3031029093',
 'MLB-3031124770',
 'MLB-3031304228',
 'MLB-2969426391',
 'MLB-3338901360',
 'MLB-2156931650',
 'MLB-2644045462',
 'MLB-2644103904',
 'MLB-2644069291',
 'MLB-2644110236',
 'MLB16082649',
 'MLB19460681',
 'MLB18719528',
 'MLB18719534',
 'MLB19542957',
 'MLB-3269832604',
 'MLB-2636283364',
 'MLB-2630247919',
 'MLB-2173128159',
 'MLB-2636319352',
 'MLB-1564104993',
 'MLB-1935535116',
 'MLB-3248018053',
 'MLB-3183188165',
 'MLB-3139072305',
 'MLB-3152222905',
 'MLB-2636264660',
 'MLB-1872855903',
 'MLB-2636258478',
 'MLB-3069795657',
 'MLB-3226349774',
 'MLB-3269771888',
 'MLB-2707401416',
 'MLB-3226355862',
 'MLB-1935536652',
 'MLB-3244826631',
 'MLB-1564103981',
 'MLB-3269796658',
 'MLB-1935619428',
 'MLB-1934511546',
 'MLB-3247934937',
 'MLB-3226406799',
 'MLB-1872838355',
 'MLB-2636270503',
 'MLB-1564102607',
 'MLB-2636277127',
 'MLB-1935553228',
 'MLB-3138085541',
 'MLB-3139118306',
 'MLB-2636293901',
 'MLB-3226426704',
 'MLB197

In [17]:
for batch in tqdm(range(0,len(asinlist))):
    print(batch)
    for val in (range(batch*50,(batch*50)+50)):
        i=asinlist[val]
        print(batch,i)
        if "-" in i:
            Url= "https://produto.mercadolibre.com.br/{}".format(i)
        else:
            Url="https://www.mercadolivre.com.br//p/{}".format(i)

        #Url = "https://produto.mercadolibre.com.br/{}".format(i)
        #Url = "https://articulo.mercadolibre.com.mx/{}".format(i)
        print(Url)
        value=''
        #s= get_session(proxies)
        r= requests.get(Url)

        soup=BeautifulSoup(r.content,'html.parser')

        try:
            main=soup.find('div',class_="ui-pdp-specs__table")
            labels=[labels.text for labels in main.find_all('th')]
            try:
                index=labels.index('Marca')
                value=soup.find_all('td')[index].text
                print(value)
            except:
                value= 'not available'
                print(value)
        except:
            pass

        try:
            main=soup.find('div',class_="ui-vpp-striped-specs")
            labels=[labels.text for labels in main.find_all('th')]
            try:
                index=labels.index('Marca')
                value=soup.find_all('td')[index].text
                print(value)
            except:
                value= 'not available'
                print(value)
        except:
            pass

        try:
            category=soup.find('div',class_="ui-pdp-breadcrumb")
            cats=[labels.text for labels in category.find_all('li')]
            print(cats)
        except:
            cats=[]

        try:
            sales=soup.find('div',class_='ui-pdp-header')
            d=sales.find('span',class_='ui-pdp-subtitle')
            sold=d.text
            print(sold)
        except:
            sold='na'
            print(sold)

        df1={'channel_sku_id': i, 'brand_name': value, 'categories': cats, 'sold':sold}
        df=df.append(df1, ignore_index = True)
    time.sleep(300)

  0%|                                                                                         | 0/2003 [00:00<?, ?it/s]


0
0 MLB-2168433399
https://produto.mercadolibre.com.br/MLB-2168433399


ConnectionError: HTTPSConnectionPool(host='produto.mercadolibre.com.br', port=443): Max retries exceeded with url: /MLB-2168433399 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002639F45E0E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
df

In [ ]:
df.info()

In [ ]:
final1=pd.merge(rawdf1,df,left_on='sku', right_on='Asin', how= 'right')
final=final1[['sku','title','Asin','Brand','Category','sold']]

In [ ]:
final

In [ ]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_mercadolibre_brazil_{}.xlsx'.format(request_no)

In [ ]:
final.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['nitin.bhayana@1digitalstack.ai','ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()

In [ ]:
final

In [ ]:
Url ='https://articulo.mercadolibre.com.mx/MLM-1422791955'

In [ ]:
r= requests.get(Url,headers=headers)
soup=BeautifulSoup(r.content,'html.parser')
main=soup.find('div',class_="ui-pdp-description")
main

In [ ]:
main

In [ ]:
labels=[labels.text for labels in main]
labels


In [ ]:
try:
            main=soup.find('div',class_="ui-pdp-specs__table")
            labels=[labels.text for labels in main.find_all('th')]
            try:
                index=labels.index('Marca')
                value=soup.find_all('td')[index].text
                print(value)
            except:
                value= 'not available'
                print(value)
        except:
            pass

In [ ]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
#from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()
import time

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r'C:\Users\Administrator\Desktop\chromedriver.exe',chrome_options = chrome_options)


In [ ]:
link=Url

In [ ]:
        print(link)
        driver.get(link)
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, locator)))
        